In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor,XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/sample_submission.csv')

In [6]:
import numpy as np

def engineer_features(df):
    """피처 엔지니어링 함수"""
    df = df.copy()
    
    # 2.1 Age
    df['age_squared'] = df['Age'] ** 2
    df['age_40_60'] = ((df['Age'] >= 40) & (df['Age'] <= 60)).astype(int)
    
    # 2.4 Cholesterol
    df['chol_capped'] = df['Cholesterol'].clip(upper=320)
    df['chol_peak'] = ((df['Cholesterol'] >= 280) & (df['Cholesterol'] <= 320)).astype(int)
    
    # 2.6 MaxHR
    df['maxhr_inv'] = 1 / (df['Max HR'] + 1)
    df['maxhr_high'] = (df['Max HR'] >= 150).astype(int)
    
    # 2.7 STdepression
    df['st_log'] = np.log1p(df['ST depression'])
    df['st_abnormal'] = (df['ST depression'] > 0).astype(int)
    
    return df

# 적용
train_df = engineer_features(train_df)
test_df = engineer_features(test_df)

# replace를 타겟 피처 이진화
train_df['Heart Disease'] = train_df['Heart Disease'].replace({'Presence': 1, 'Absence': 0})
train_df['Heart Disease'] = train_df['Heart Disease'].astype(int)

In [8]:
train_df.columns

Index(['id', 'Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol',
       'FBS over 120', 'EKG results', 'Max HR', 'Exercise angina',
       'ST depression', 'Slope of ST', 'Number of vessels fluro', 'Thallium',
       'Heart Disease', 'age_squared', 'age_40_60', 'chol_capped', 'chol_peak',
       'maxhr_inv', 'maxhr_high', 'st_log', 'st_abnormal'],
      dtype='str')